In [1]:
import csv
import zipfile
import random

local = False
root_path = "./" if local else "/tmp/moviele-main/"

if not local:
    !wget --no-check-certificate "https://github.com/andynosbig/moviele/archive/refs/heads/main.zip" -O "/tmp/main.zip"
    zip_ = zipfile.ZipFile('/tmp/main.zip', 'r')
    zip_.extractall('/tmp')
    zip_.close()

films_by_title = {}
with open(f"{root_path}combined.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        films_by_title[row['title']] = row
        films_by_title[row['title']].pop('id')
        
        films_by_title[row['title']]['genres'] = row['genres'].split(',')
        films_by_title[row['title']]['principals'] = row['principals'].split(',')

principalNames = {}
with open(f"{root_path}principals.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        principalNames[row['id']] = row['name']

genres = []
with open(f"{root_path}genres.csv", 'r') as genre_csv:
    genres = genre_csv.read().split(',')

film_by_genre = {}
for genre in genres:
    film_by_genre[genre] = {t:f for t,f in films_by_title.items() if genre in f['genres']}


In [18]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

import ipywidgets as widgets

class Target:
    film = None
    guesses = 0

class Genre():
    genre = None
    
def get_random_item(list_):
    return list(list_)[random.randint(0, len(list_)-1)]

def calc_list_similarity(listA, listB):
    return 100 * (len(set(listA)&set(listB)) / len(set(listA)|set(listB))), list(set(listA)&set(listB))
    
def run():
    selected_genre = Genre()
    selected_genre.genre = genres[0]
    

    out = widgets.Output(layout={'border': '1px solid black'})
    target = Target()

    def test_guess(x):
        if x not in film_by_genre[selected_genre.genre]:
            return

        target.guesses += 1
        if film_by_genre[selected_genre.genre][x] == target.film:
            with out:
                print(f"\n{target.guesses} {x}\t\t'Success!'")
                return

        
        guess_film = film_by_genre[selected_genre.genre][x]
        genre_percentage, genre_overlap = calc_list_similarity(guess_film['genres'], target.film['genres'])
        principals_percentage, principals_overlap = calc_list_similarity(guess_film['principals'], target.film['principals'])
        principal_name_overlap = [principalNames[x] for x in principals_overlap]
        year_diff = int(guess_film['startYear']) - int(target.film['startYear'])

        result = f"\n{target.guesses} {x} - \n\tprincipal overlap\t{','.join(principal_name_overlap)}  \n\tgenre overlap\t\t{','.join(genre_overlap)} \n\tRelease date\t\t{abs(year_diff)} {'earlier' if year_diff < 0 else 'later'} than target"
        with out:
            print(result)

    combo = widgets.Combobox()
    interact(test_guess, x=combo)
    def setup_combo():
        combo.options=sorted(film_by_genre[selected_genre.genre].keys())
    setup_combo()

    def get_hint(event):
        with out:
            print(f"hint,  {principalNames[get_random_item(target.film['principals'])]} is involved in this film")

    def set_new_target(event):
        target.film = get_random_item(film_by_genre[selected_genre.genre].values())
        target.guesses = 0
        out.clear_output()
        get_hint(None)

    genre_rbs = widgets.RadioButtons(options=genres, description="genres", disabled=False)
    def choose_genre(genre):
        selected_genre.genre = genre
        set_new_target(None)
        setup_combo()

    widgets.interact(choose_genre, genre = genre_rbs)

    new_target_button = widgets.Button(description = 'Reset Target Film')
    get_answer_button = widgets.Button(description = 'Show Answer')
    get_hint_button = widgets.Button(description = 'Hint')
    
    get_hint_button.on_click(get_hint)

    set_new_target(None)
    new_target_button.on_click(set_new_target)

    def show_answer(event):
        with out:
            print("Answer: ", target.film['title'])

    get_answer_button.on_click(show_answer)
    display(widgets.HBox((new_target_button, get_answer_button, get_hint_button)))   

    display(out)
    #interact(test_guess, x=films)
    interact


run()

interactive(children=(Combobox(value='', description='x'), Output()), _dom_classes=('widget-interact',))

interactive(children=(RadioButtons(description='genres', options=('Sport', 'Horror', 'Biography', 'Action', 'C…

Output(layout=Layout(border='1px solid black'))